In [1]:
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
from keras.src.applications.efficientnet import EfficientNetB0
from keras.src.legacy.preprocessing.image import NumpyArrayIterator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score, precision_score, \
    recall_score, f1_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

In [2]:
# Hyperparameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Define paths
data_dir = 'data'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [3]:
# Data Augmentation
data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
)

# Train Data
train_data = data_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

# Validation Data
val_data = data_gen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

# Test Data
test_data = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Calculate Class Weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights = dict(enumerate(class_weights))


Found 7000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 3000 images belonging to 10 classes.


In [4]:
x_batch, y_batch = next(iter(train_data))
print("X batch shape:", x_batch.shape)
print("Y batch shape:", y_batch.shape)
print("Data type of X:", x_batch.dtype)
print("Data type of Y:", y_batch.dtype)

X batch shape: (32, 224, 224, 3)
Y batch shape: (32, 10)
Data type of X: float32
Data type of Y: float32


In [ ]:
# Define CNN Model
cnn_model = Sequential([
    tf.keras.layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Flatten(),
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')
])

cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
model_checkpoint = ModelCheckpoint('best_cnn_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Train CNN Model
cnn_history = cnn_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    # callbacks=[early_stopping, lr_scheduler],
    callbacks=[early_stopping, lr_scheduler, model_checkpoint],
    # class_weight=class_weights
)

C:\Users\viraj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
116/219 ━━━━━━━━━━━━━━━━━━━━ 1:34 920ms/step - accuracy: 0.4309 - loss: 3.2429

In [ ]:
# Fine-Tuned ResNet50 Model
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
resnet_base.trainable = True  # Fine-tune

resnet_model = Sequential([
    resnet_base,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')
])

resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train ResNet50 Model
resnet_history = resnet_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stopping, lr_scheduler],
    # class_weight=class_weights
)

In [ ]:
# # New EfficientNet Model
# eff_model = Sequential([
#     EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
#     GlobalAveragePooling2D(),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(train_data.num_classes, activation='softmax')
# ])
# 
# eff_model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )
# 
# # Train EfficientNet Model
# eff_history = eff_model.fit(
#     train_data,
#     validation_data=val_data,
#     epochs=EPOCHS,
#     callbacks=[early_stopping, lr_scheduler],
#     class_weight=class_weights
# )

In [ ]:
# Save Models
cnn_model.save('cnn_model.keras')
resnet_model.save('resnet50_model.keras')

In [ ]:
# Evaluate Model Function
def evaluate_model(model, data, model_name="Model"):
    loss, accuracy = model.evaluate(data)
    print(f"{model_name} Test Accuracy: {accuracy:.2f}")
    
    # Generate Predictions
    data.reset()
    predictions = model.predict(data)
    y_pred = np.argmax(predictions, axis=1)
    y_true = data.classes

    # Classification Metrics
    print(f"\n{model_name} Classification Report:")
    print(classification_report(y_true, y_pred, target_names=list(data.class_indices.keys())))

    # Compute Evaluation Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')  # Weighted for imbalanced classes
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_true, predictions, multi_class='ovr')  # For multi-class problems
    
    # Print Metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=list(data.class_indices.keys()),
                yticklabels=list(data.class_indices.keys()))
    plt.title(f"{model_name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


# Evaluate All Models
evaluate_model(cnn_model, test_data, "CNN")
evaluate_model(resnet_model, test_data, "ResNet50")
# evaluate_model(eff_model, val_data, "EfficientNetB0")

In [ ]:
def tr_plot(tr_data, start_epoch):
    # Extract metrics from training history
    train_acc = tr_data.history['accuracy']
    train_loss = tr_data.history['loss']
    val_acc = tr_data.history['val_accuracy']
    val_loss = tr_data.history['val_loss']

    # Calculate epoch count and epoch range
    total_epochs = len(train_acc) + start_epoch
    epochs = list(range(start_epoch + 1, total_epochs + 1))

    # Identify the best epochs based on validation loss and accuracy
    best_val_loss_epoch = np.argmin(val_loss)  # Epoch with the lowest validation loss
    best_val_loss = val_loss[best_val_loss_epoch]

    best_val_acc_epoch = np.argmax(val_acc)  # Epoch with the highest validation accuracy
    best_val_acc = val_acc[best_val_acc_epoch]

    # Plot style
    plt.style.use('fivethirtyeight')

    # Labels for best epochs
    loss_label = f"Best Epoch (Loss): {best_val_loss_epoch + 1 + start_epoch}"
    acc_label = f"Best Epoch (Accuracy): {best_val_acc_epoch + 1 + start_epoch}"

    # Create subplots for loss and accuracy
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))

    # Plot training and validation loss
    axes[0].plot(epochs, train_loss, 'r', label='Training Loss')
    axes[0].plot(epochs, val_loss, 'g', label='Validation Loss')
    axes[0].scatter(best_val_loss_epoch + 1 + start_epoch, best_val_loss, s=150, c='blue', label=loss_label)
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()

    # Plot training and validation accuracy
    axes[1].plot(epochs, train_acc, 'r', label='Training Accuracy')
    axes[1].plot(epochs, val_acc, 'g', label='Validation Accuracy')
    axes[1].scatter(best_val_acc_epoch + 1 + start_epoch, best_val_acc, s=150, c='blue', label=acc_label)
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

# Call the function with the correct parameters
tr_plot(cnn_history, 0)
tr_plot(resnet_history, 0)